<table>
    <td>
        <img src="https://ciadaconcepcao.com.br/wp-content/uploads/2017/01/innohub-270x270.png" style="width: 100px">
    </td>
    <td>
        <img src="https://i.ibb.co/vJ858Kg/ets-logo2.png" style="width: 200px">
    </td>
</table>


# <center>Treinamento Iot - Módulo II</center>
## <center>Aula 4 - Pandas - parte 3</center>



<img src="https://media.giphy.com/media/EatwJZRUIv41G/giphy.gif">

Vamos continuar usando o __Pandas__ para trabalhar nosso dados.

Vamos abrir o dataset que foi salvo no final da aula anterior.


In [2]:
import pandas as pd
titanic = pd.read_csv("data/titanic_1_aula.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/titanic_1_aula.csv'

## Tratando valores nulos

#### Porque tratar valores nulos?

Valores nulos atrapalham a utilização de algoritmos de ML e podem atrapalhar algumas análises também. 


In [ ]:
titanic.head(20)

Assim a gente consegue ver todos os dados nulos por coluna:

In [ ]:
titanic.isnull().sum()

Como funciona? O que acontece é que se somarmos todos os dados True/False ele vai dar a quantidade de True (que equivale a 1)

In [ ]:
titanic.Age.isnull()

### Podemos tratar valores nulos por duas abordagens

### 1ª: Apagando Nulos

In [ ]:
titanic.Age.head(10)

In [ ]:
idades_nao_nulas = titanic.Age.dropna()
idades_nao_nulas.head(10)

In [ ]:
qnt_idades_nulas_dropna = idades_nao_nulas.isnull().sum()
print("Quantidade de valores nulos: {}".format(qnt_idades_nulas_dropna))

Size x Count :
 - `size` - o tamanho total da série
 - `count` - a quantidade de valores não nulos na série

In [ ]:
titanic.Age.size

In [ ]:
titanic.Age.count()

In [ ]:
# idades_nao_nulas.size é igual a titanic.Age.count()
idades_nao_nulas.size

__Atribuir uma série com os valores nulos apagados a uma coluna do DataFrame não faz sentido! A coluna vai voltar a possuir valores nulos__


Como o tamanho da Série vai ser diferente da quantidade de linhas do Dataframe, os valores que faltam na série serão colocados como __nulos__ conforme o índice. 

In [ ]:
titanic["AgeNotNull"] = idades_nao_nulas

In [ ]:
titanic[["AgeNotNull","Age"]].count()

Isso acontece por causa do índice

In [ ]:
idades_nao_nulas.head(10) # procure o 5

### 2ª: Preenchendo com algum valor

Se desejamos preencher os valores nulos com algum valor, podemos usar o __fillna__.

__Nesse caso, se criarmos uma coluna com o resultado do `fillna`, ela realmente não terá mais valores nulos!__

In [ ]:
# Preenchendo as idades nulas com -1
serie_idade_sem_nulo = titanic.Age.fillna(-1)
serie_idade_sem_nulo.isnull().sum()

In [ ]:
serie_idade_sem_nulo.count()

In [ ]:
titanic["AgeFillNa-1"] = serie_idade_sem_nulo

In [ ]:
titanic["AgeFillNa-1"].head(10)

##### Preencher com `-1` não parece ser a melhor escolha.

## Uma técnica muito usada é trocar o valor dos dados númericos é usar a média
### Vamos preencher os valores de idade nulo com a médias das idades

![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/desafio.jpg)

In [ ]:
titanic["AgeFillNaMean"] = titanic.Age.fillna(titanic.Age.mean())

In [ ]:
titanic[['Age','AgeFillNaMean']].head(10)

## Agrupando

Podemos agrupar os dados baseados em uma coluna.O agrupamento aplicando uma função nos dados que estão no mesmo grupo

O `groupby` seguido de uma operação funciona da seguinte forma:
 1. Agrupa as linhas com valores iguais na coluna do agrupamento. [no código a coluna do agrupamente é colocada `groupby(COLUNA_DO_AGRUPAMENTO)`]
 2. Aplica uma função em cada grupo gerado no passo (1) que transforme todas as linhas em apenas um valor.
 3. Na imagem abaixo, estamos aplicando o groupby na coluna "X", e aplicando uma função de MÉDIA na coluna "Y". <br>
     `df.groupby("X")["Y"].mean()`
 
![groupby](https://databootcamp.nyc3.digitaloceanspaces.com/img/groupby.png)

In [ ]:
# Agrupando pessoas por sexo
pessoas_por_sexo = titanic.groupby("Sex").count()
pessoas_por_sexo.head()

O resultado foi um Dataframe, que mostra a quantidade de dados não nulos em cada coluna dos grupos das cabines.

Se quisermos apenas o valor de pessoas por cabine, basta selecionarmos uma coluna que tenha todos os dados não nulos (Já que vimos que o `count()` conta apenas os nãos nulos)

In [ ]:
# Quantas pessoas tem em cada cabine?
pessoas_por_cabine = titanic.groupby("Cabin")["PassengerId"].count()
pessoas_por_cabine.head(20)

### Outra maneira é substituir o `count` por `size`, então podemos contar a quantidade de elementos em qualquer coluna, mesmo com valores nulos

In [ ]:
# A coluna Age tem valores nulos. Podemos usar o "size" para contar por ela
pessoas_por_cabine = titanic.groupby("Cabin")["Age"].size()
pessoas_por_cabine.head(20)

#### Outra maneira de agrupar é pela soma
### Qual a idade somada das pessoas em cada cabine?

In [ ]:
soma_idade = titanic.groupby("Cabin")["Age"].sum()
soma_idade.head(20)

### Também podemos agrupar pela média

In [ ]:
# Qual a idade média das pessoas de cada sexo?
media_idade = titanic.groupby("Sex")["Age"].mean()
media_idade.head(20)

#### Existe uma maneira mais livre para agruparmos. Podemos usar o __apply!__

### Usando o apply para contar os passageiros por cabine

In [ ]:
def contar_pessoas(registros_agrupados):
    return registros_agrupados["PassengerId"].size


pessoas_por_cabine_apply = titanic.groupby("Cabin").apply(contar_pessoas)
pessoas_por_cabine_apply.head(10)

### Vamos fazer o agrupamento por cabine somando as idades usando o apply

![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/desafio.jpg)

In [ ]:
def contar_pessoas(registros_agrupados):
    return registros_agrupados.Age.sum()


soma_idade_apply_certo = titanic.groupby("Cabin").apply(contar_pessoas)
soma_idade_apply_certo.describe()

### Vamos voltar ao caso de contar quantos homens e mulheres temos

In [ ]:
# Usando o count()
soma_idade_apply_certo = titanic.groupby("Sex")["PassengerId"].count()
soma_idade_apply_certo

In [ ]:
# Usando apply
def count_sex(registros):
    return registros["Sex"].count()

titanic.groupby("Sex").apply(count_sex)

### Desafio
Lembram que não ficou muito bom quando colocamos a média geral da idade nas idades nulas?

### Vamos tentar melhorar a média das idades!


**Vamos preencher os nulos de maneira que se o passageiro for mulher esse recebe a idade media das mulheres.
Se o passageiro for homem ele recebe a idade média dos homens**


![alt text](https://databootcamp.nyc3.digitaloceanspaces.com/img/desafio.jpg)

In [ ]:
media_mulher = titanic[titanic.Sex == "female"].Age.mean()
media_homen = titanic[titanic.Sex == "male"].Age.mean()

print("""
Media de idade das mulheres: {}
Media de idade dos homens: {}""".format(media_mulher,media_homen))

In [ ]:
def preenche_idade(linha):
    idade = linha["Age"]
    sexo = linha["Sex"]
    
    if pandas.isnull(idade): #funcao padrão do pandas para verificar se na linha tem dado nulo
        
        if sexo == "female":
            return media_mulher
        else:
            return media_homen
        
    else:
        return idade
    

idades_preenchidas = titanic.apply(preenche_idade,axis=1)

print("""
Quantidade de nulos após o preenchimento: {}
""".format(idades_preenchidas.isnull().sum()))       

In [ ]:
# Vamos criar a coluna idade preenchida pra usar o resultado do apply
titanic["AgeFillNaSexMean"] = idades_preenchidas

In [ ]:
titanic[["Age", "AgeFillNaSexMean"]]

# Salvando no trabalho :D

In [ ]:
titanic.to_csv("titanic_2_aula.csv", index=False)

In [ ]:
titanic.info()